__Predictions are based the best model.__

In [1]:
import warnings
warnings.filterwarnings('ignore')
from keras.models import Sequential
from keras.layers import Dense,Dropout
def mlp(dropout_rate=0.0,activation1='relu', activation2='relu', activation3='relu', activation4='relu',
                       activation5='relu',activation6='relu'):
    model = Sequential()
    model.add(Dense(1024, input_dim=len(X.columns), kernel_initializer='normal', activation=activation1))
    model.add(Dropout(dropout_rate))
    model.add(Dense(512, kernel_initializer='normal', activation=activation2))
    model.add(Dropout(dropout_rate))
    model.add(Dense(256, kernel_initializer='normal', activation=activation3))
    model.add(Dropout(dropout_rate))
    model.add(Dense(128, kernel_initializer='normal', activation=activation4))
    model.add(Dropout(dropout_rate))
    model.add(Dense(64, kernel_initializer='normal', activation=activation5))
    model.add(Dropout(dropout_rate))
    model.add(Dense(32, kernel_initializer='normal', activation=activation6))
    model.add(Dense(4, kernel_initializer='normal', activation='relu'))
    # Compile model
    model.compile(loss='mae', optimizer='adam', metrics=['mae'])
    return model

Using TensorFlow backend.


__Transforming the given test set in the same way as the training set.__

In [2]:
import  pandas as pd
!pip install holidays
import holidays
test_df = pd.read_csv('test_data.csv',index_col=0,parse_dates=[1])
df = pd.DataFrame()
de_by_holidays = holidays.CountryHoliday('DE',prov='by')
df['month'] = test_df['ride_departure'].dt.month
df['day_of_year'] = test_df['ride_departure'].dt.dayofyear
df['hour'] = test_df['ride_departure'].dt.hour
df['minute'] = test_df['ride_departure'].dt.minute
df['day_of_week'] = test_df['ride_departure'].dt.dayofweek
df['holiday'] = test_df['ride_departure'].dt.date.apply(lambda x : 1 if x in de_by_holidays else 0)
df['direction'] = test_df['direction']
df = pd.get_dummies(df,prefix='route',columns=['direction']) 
test_df.head()

,ride_departure,capacity,direction
index,,,
19219,2017-05-01 07:00:00,75,B->A
19220,2017-05-01 08:30:00,75,A->B
19221,2017-05-01 09:00:00,75,B->A
19222,2017-05-01 10:00:00,75,B->A
19223,2017-05-01 10:30:00,75,A->B


__Using the pickled scaler once again to transform the data.__

In [3]:
import joblib
scaler = joblib.load('scaler.pkl')
df.loc[:,'month':'day_of_week'] = scaler.transform(df.loc[:,'month':'day_of_week'])
df['capacity'] = test_df['capacity']
df.head()

,month,day_of_year,hour,minute,day_of_week,holiday,route_A->B,route_B->A,capacity
index,,,,,,,,,
19219,-0.483933,-0.642168,-1.345848,-1.874368,-1.612973,1,0,1,75
19220,-0.483933,-0.642168,-1.148080,0.110620,-1.612973,1,1,0,75
19221,-0.483933,-0.642168,-0.950313,-1.874368,-1.612973,1,0,1,75
19222,-0.483933,-0.642168,-0.752546,-1.874368,-1.612973,1,0,1,75
19223,-0.483933,-0.642168,-0.752546,0.110620,-1.612973,1,1,0,75


__Saving the predictions in the specified format.__

In [4]:
import warnings
warnings.filterwarnings('ignore')
from keras.models import load_model
path = 'models'
model = load_model(path +'/model-5.8806.hdf5')
predictions_df = pd.DataFrame(model.predict(df),columns=['tickets_9_eur','tickets_12_eur','tickets_15_eur','tickets_19_eur'])
predictions_df.index = test_df.index
predictions_df = predictions_df.round(1)
predictions_df.to_csv('predictions.csv')
predictions_df.head()

W0902 13:31:27.129225 4614784448 deprecation_wrapper.py:119] From /Users/raghuramkalyanam/anaconda/lib/python3.6/site-packages/keras/backend/tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0902 13:31:27.158071 4614784448 deprecation_wrapper.py:119] From /Users/raghuramkalyanam/anaconda/lib/python3.6/site-packages/keras/backend/tensorflow_backend.py:4115: The name tf.random_normal is deprecated. Please use tf.random.normal instead.

W0902 13:31:27.180958 4614784448 deprecation_wrapper.py:119] From /Users/raghuramkalyanam/anaconda/lib/python3.6/site-packages/keras/backend/tensorflow_backend.py:131: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W0902 13:31:27.182183 4614784448 deprecation_wrapper.py:119] From /Users/raghuramkalyanam/anaconda/lib/python3.6/site-packages/keras/backend/tensorflow_backend.py:133: The name tf.placeholder_with_default is deprecated. Please use tf.

,tickets_9_eur,tickets_12_eur,tickets_15_eur,tickets_19_eur
index,,,,
19219,1.0,13.2,4.7,0.0
19220,7.5,9.8,0.0,0.0
19221,1.3,13.3,9.9,0.0
19222,1.4,17.9,9.9,0.0
19223,12.1,0.4,0.0,0.0
